<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/7-deep-dive-into-keras/01_keras_model_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Keras model fundamentals

There are three APIs for building models in Keras:

* The Sequential model, the most approachable API—it’s basically a Python list. As such, it’s limited to simple stacks of layers.
* The Functional API, which focuses on graph-like model architectures. It represents
a nice mid-point between usability and flexibility, and as such, it’s the
most commonly used model-building API.
* Model subclassing, a low-level option where you write everything yourself from
scratch. This is ideal if you want full control over every little thing. However, you
won’t get access to many built-in Keras features, and you will be more at risk of
making mistakes.

<img src='https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/7-deep-dive-into-keras/images/1.png?raw=1' width='600'/>

##Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random
import string
import re

import numpy as np

##Sequential model

The simplest way to build a Keras model is to use the Sequential model.

In [2]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")                      
])

Note that it’s possible to build the same model incrementally via the `add()` method,
which is similar to the `append()` method of a Python list.

In [3]:
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

As such, the preceding Sequential model does not have any weights, until you actually call it on some data, or call its
method with an input shape `build()`.

In [4]:
# At that point, the model isn’t built yet
# model.weights

```log
ValueError: Weights for model sequential_1 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.
```

Builds the model—now the model will expect samples of shape (3,). The
None in the input shape signals that the batch size could be anything.

In [5]:
model.build(input_shape=(None, 3))
# Now you can retrieve the model’s weights
model.weights

[<tf.Variable 'dense_2/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[-0.03376526,  0.24571598, -0.09835906, -0.00932831,  0.08157629,
          0.05217475, -0.20005333, -0.1406315 ,  0.12143812, -0.26803538,
         -0.2816533 , -0.03278193, -0.27682567,  0.0872719 , -0.15364805,
         -0.1635248 , -0.02422559,  0.27696687,  0.0916079 , -0.02749529,
          0.18096241, -0.27949056,  0.2978536 ,  0.08078364,  0.1801078 ,
          0.15023121, -0.10418415, -0.25423342, -0.1823534 ,  0.19416669,
          0.01058289, -0.0026412 ,  0.11240751,  0.08405375, -0.00205353,
         -0.284914  ,  0.01765954, -0.07185607,  0.01497647,  0.29167515,
         -0.03647739, -0.00298688,  0.01986891, -0.18263593, -0.28273052,
         -0.28212577, -0.04771101,  0.06716427, -0.04724547, -0.09143116,
          0.10478953,  0.21232557,  0.25604987, -0.19146061, -0.03347814,
          0.05119607,  0.05474174, -0.29161963, -0.25620157,  0.17473492,
         -0.27108172, -0.11325027,  0.26775

After the model is built, you can display its contents via the
method, which `summary()` comes in handy for debugging.

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                256       
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


As you can see, this model happens to be named “sequential_1.” You can give names
to everything in Keras—every model, every layer.

In [7]:
model = keras.Sequential(name="my_example_model")
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))

model.build((None, 3))
model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
 my_last_layer (Dense)       (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


But you can’t print a summary until the model is built! 

There’s actually a way to have your Sequential
built on the fly: just declare the shape of the model’s inputs in advance. You can do this via the Input class.

In [8]:
model = keras.Sequential(name="my_example_model")
model.add(keras.Input(shape=(3, )))
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))

Now you can use `summary()` to follow how the output shape of your model changes as
you add more layers

In [9]:
model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))

model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
 my_last_layer (Dense)       (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


This is a pretty common debugging workflow when dealing with layers that transform
their inputs in complex ways, such as the convolutional layers.

##Functional API

The Sequential model is easy to use, but its applicability is extremely limited: it can
only express models with a single input and a single output, applying one layer after
the other in a sequential fashion.

Let’s start with something simple.

In [11]:
inputs = keras.Input(shape=(3, ), name="my_input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)

model = keras.Model(inputs=inputs, outputs=outputs)

This inputs object holds information about the shape and dtype of the data that the
model will process:

In [12]:
inputs.shape

TensorShape([None, 3])

In [13]:
inputs.dtype

tf.float32

We call such an object a symbolic tensor. It doesn’t contain any actual data, but it
encodes the specifications of the actual tensors of data that the model will see when
you use it. It stands for future tensors of data.

All Keras layers can be called both on real tensors of data and on these symbolic tensors.

In [14]:
features.shape

TensorShape([None, 64])

In [15]:
features.dtype

tf.float32

Here’s the summary of our model:

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_4 (Dense)             (None, 64)                256       
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


###Multi-input and Multi-output